# Lesson 6: ADK + MCP Integration - The Complete AI Agent Stack

## ⚠️  IMPORTANT: LOCAL EXECUTION REQUIRED

**This notebook requires LOCAL execution** (Jupyter Lab, VS Code, PyCharm, etc.)

**Why not Colab?**  
MCP servers communicate via stdio (standard input/output), which is not compatible with Google Colab's custom I/O system. You'll encounter `UnsupportedOperation: fileno` errors if you try to run this in Colab.

**Options:**
1. **Run locally** - Download this notebook and run on your machine (recommended)
2. **Adapt** - Modify to use HTTP-based MCP servers (advanced)

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:

1. **Understand** the modern AI agent architecture stack
2. **Create** MCP servers that expose data and tools
3. **Connect** ADK agents to MCP servers using McpToolset
4. **Build** production-grade systems combining ADK orchestration + MCP data connectivity
5. **Design** composable, modular AI architectures
6. **Deploy** the complete stack locally

## 🎓 Where We Are Now

**Lessons 1-5: ADK**
- ✅ Basic agents with LLMs
- ✅ Function calling and tools
- ✅ Multi-agent coordination
- ✅ Workflow patterns (Sequential, Parallel, Loop)
- ✅ Hybrid architectures (rules + LLMs)

**Lesson 6: The Complete Stack** (
- 🚀 Bringing it all together!
- 🔗 Connecting ADK to real data sources via MCP
- 🏗️ Building production-ready architectures

---

## 🏗️ AI Agent Stack

### Three-Layer Architecture

```
┌─────────────────────────────────────────────┐
│  Layer 1: LLM Provider (OpenAI)             │  ← Reasoning & Intelligence
│  - gpt-5-nano, gpt-4o, etc.                 │
│  - Model-agnostic (swap providers easily)   │
└─────────────────────────────────────────────┘
                    ▲
                    │
┌─────────────────────────────────────────────┐
│  Layer 2: ADK (Agent Development Kit)       │  ← Orchestration & Coordination
│  - Multi-agent coordination                 │
│  - Workflow patterns                        │
│  - Tool management                          │
│  - Session handling                         │
└─────────────────────────────────────────────┘
                    ▲
                    │
┌─────────────────────────────────────────────┐
│  Layer 3: MCP (Model Context Protocol)      │  ← Data Connectivity
│  - Standardized tool/data interface         │
│  - Connect to: databases, APIs, files       │
│  - Reusable across AI frameworks            │
└─────────────────────────────────────────────┘
                    ▲
                    │
┌─────────────────────────────────────────────┐
│  Data Sources                               │
│  - Ticket DB, Knowledge Base, Monitoring    │
└─────────────────────────────────────────────┘
```

### Why This Architecture?

**Separation of Concerns:**
- 🧠 **LLM**: Provides reasoning (model-agnostic)
- 🎭 **ADK**: Orchestrates agents and workflows
- 🔌 **MCP**: Standardizes data connectivity

**Benefits:**
- ✅ **Composability**: Mix and match components
- ✅ **Reusability**: MCP servers work with any MCP client
- ✅ **Maintainability**: Change one layer without affecting others
- ✅ **Scalability**: Add new data sources easily



We'll build a complete IT support system:

**MCP Servers (Data Layer):**
1. 🎫 **Ticket Database Server**: CRUD operations for support tickets
2. 📚 **Knowledge Base Server**: Search documentation and articles
3. 🖥️ **System Monitoring Server**: Check system health and metrics

**ADK Agents (Orchestration Layer):**
1. 🤖 **Support Coordinator**: Routes requests to specialists
2. 🎫 **Ticket Specialist**: Manages tickets via MCP
3. 📖 **Knowledge Specialist**: Searches docs via MCP
4. 🔧 **System Specialist**: Monitors systems via MCP

**OpenAI (Reasoning Layer):**
- All agents use gpt-5-nano for decision-making

---

## 🔧 Part 1: Environment Setup

In [ ]:
# Install required packages
!pip install -q google-adk litellm openai python-dotenv nest-asyncio mcp httpx

print("✅ Packages installed successfully!")
print("   - google-adk: Agent orchestration")
print("   - mcp: Model Context Protocol")
print("   - litellm: OpenAI integration")
print("")
print("⚠️  IMPORTANT NOTE ABOUT GOOGLE COLAB:")
print("   MCP servers use stdio (standard input/output) for communication.")
print("   This does NOT work in Google Colab due to Colab's custom I/O handling.")
print("")
print("   This notebook is designed for LOCAL execution (Jupyter, VS Code, etc.)")
print("   To run in Colab, you would need to use HTTP-based MCP servers instead.")
print("")
print("   For learning purposes, you can:")
print("   1. Run this notebook locally (recommended)")
print("   2. Read through the code to understand the architecture")
print("   3. Adapt for HTTP-based MCP servers for Colab")

In [ ]:
# Core ADK imports
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.models.lite_llm import LiteLlm
from google.genai import types

# MCP integration imports - using new class name
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset  # Note: McpToolset (not MCPToolset)
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

# System imports
import os
import sys
import asyncio
import subprocess
import time
import signal
from pathlib import Path
import warnings

# Enable nested asyncio for Colab
import nest_asyncio
nest_asyncio.apply()

# Suppress stdout/stderr warnings from MCP tools (redirect temporarily)
import contextlib
from io import StringIO

# Filter Python warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='google.adk.tools.mcp_tool')
warnings.filterwarnings('ignore', message='.*EXPERIMENTAL.*')
warnings.filterwarnings('ignore', message='.*auth_config.*')

# Suppress MCP tool authentication messages (they print to stderr directly)
os.environ['PYTHONWARNINGS'] = 'ignore'

print("✅ Imports successful!")
print("   Key import: McpToolset for connecting to MCP servers")
print("   MCP integration: google.adk.tools.mcp_tool")
print("   Note: Auth warnings suppressed (MCP tools work without auth for local servers)")

In [ ]:
# Configure OpenAI API key
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    from getpass import getpass
    print("💡 To use Colab secrets: Go to 🔑 (left sidebar) → Add new secret → Name: OPENAI_API_KEY")
    OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

if not OPENAI_API_KEY or OPENAI_API_KEY.strip() == "":
    raise ValueError("❌ ERROR: No API key provided!")

print("✅ Authentication configured!")

# Model configuration
OPENAI_MODEL = "gpt-5-nano"  # Using gpt-5-nano for cost efficiency

print(f"\n🤖 Model: {OPENAI_MODEL}")
print(f"🏗️  Architecture: OpenAI (reasoning) + ADK (orchestration) + MCP (data)")

---

## 📦 Part 2: Our MCP Servers

We've created 3 MCP servers for IT support:

1. **ticket_mcp_server.py**
   - Tools: `get_ticket`, `list_tickets`, `create_ticket`, `update_ticket`, `search_tickets`
   - Data: In-memory ticket database

2. **knowledge_mcp_server.py**
   - Tools: `search_knowledge_base`, `get_article`, `list_articles`, `get_popular_articles`
   - Data: IT support documentation and guides

3. **system_monitoring_mcp_server.py**
   - Tools: `check_system_health`, `list_all_systems`, `get_system_metrics`, `ping_system`, `get_system_logs`, `get_alerts`
   - Data: System health and monitoring data



In [ ]:
# Ensure MCP server files are in the same directory as this notebook
from pathlib import Path

MCP_SERVERS = [
    "ticket_mcp_server.py",
    "knowledge_mcp_server.py",
    "system_monitoring_mcp_server.py"
]

print("🔍 Checking for MCP server files...\n")

# Verify files exist locally
all_found = True
for server_file in MCP_SERVERS:
    if Path(server_file).exists():
        print(f"✅ {server_file}")
    else:
        print(f"❌ {server_file} not found")
        all_found = False

if all_found:
    print("\n✅ All MCP servers ready to start!")
else:
    print("\n⚠️ Missing files detected!")
    print("📥 Please ensure all three .py files are in the same directory as this notebook")

---
## Part 3: Creating Specialists

### 3.1: Ticket Management Specialist

In [ ]:
# Create McpToolset for ticket database
ticket_toolset = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="python",
            args=["ticket_mcp_server.py"]
        )
    )
)

# Create ticket specialist agent
ticket_specialist = LlmAgent(
    model=LiteLlm(model=f"openai/{OPENAI_MODEL}"),
    name="ticket_specialist",
    tools=[ticket_toolset],
    instruction="""
    You are a Ticket Management Specialist for IT support.

    YOUR CAPABILITIES:
    You can access the ticket database through MCP server tools:
    - Get ticket details by ID
    - List and filter tickets (by status, priority, team)
    - Create new tickets
    - Update ticket status and details
    - Search tickets by keywords

    YOUR ROLE:
    - Help users find and manage support tickets
    - Create tickets for new issues
    - Update ticket status as issues are resolved
    - Provide ticket summaries and analytics

    Always use the available tools to access real ticket data.
    Be concise and helpful in your responses.
    """
)

print("✅ Ticket Specialist Agent created!")
print(f"   Model: {OPENAI_MODEL}")
print(f"   Tools: Connected to Ticket Database MCP Server")
print(f"   Capabilities: CRUD operations on tickets")

### 3.2: Knowledge Base Specialist

In [ ]:
# Create McpToolset for knowledge base
knowledge_toolset = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="python",
            args=["knowledge_mcp_server.py"]
        )
    )
)

# Create knowledge specialist agent
knowledge_specialist = LlmAgent(
    model=LiteLlm(model=f"openai/{OPENAI_MODEL}"),
    name="knowledge_specialist",
    tools=[knowledge_toolset],
    instruction="""
    You are a Knowledge Base Specialist for IT support.

    YOUR CAPABILITIES:
    You can access the knowledge base through MCP server tools:
    - Search documentation by keywords
    - Retrieve specific articles by ID
    - List articles by category
    - Find popular/helpful articles

    YOUR ROLE:
    - Help users find relevant documentation
    - Provide step-by-step guides for common issues
    - Recommend helpful articles
    - Answer questions using knowledge base content

    When users ask about IT issues:
    1. Search for relevant articles
    2. Retrieve full article content if needed
    3. Provide clear, actionable guidance
    4. Include article IDs for reference

    Always search the knowledge base first before answering.
    """
)

print("✅ Knowledge Specialist Agent created!")
print(f"   Model: {OPENAI_MODEL}")
print(f"   Tools: Connected to Knowledge Base MCP Server")
print(f"   Capabilities: Search and retrieve documentation")

### 3.3: System Monitoring Specialist

In [ ]:
# Create McpToolset for system monitoring
monitoring_toolset = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="python",
            args=["system_monitoring_mcp_server.py"]
        )
    )
)

# Create system specialist agent
system_specialist = LlmAgent(
    model=LiteLlm(model=f"openai/{OPENAI_MODEL}"),
    name="system_specialist",
    tools=[monitoring_toolset],
    instruction="""
    You are a System Monitoring Specialist for IT support.

    YOUR CAPABILITIES:
    You can monitor systems through MCP server tools:
    - Check system health and status
    - List all monitored systems
    - Get detailed metrics (CPU, memory, disk, network)
    - Ping systems to test connectivity
    - Retrieve system logs
    - Get active alerts

    YOUR ROLE:
    - Monitor system health
    - Diagnose performance issues
    - Identify system problems
    - Provide recommendations

    When asked about system issues:
    1. Check system health first
    2. Get relevant metrics if needed
    3. Check logs for errors
    4. Provide clear diagnosis and recommendations

    Alert Priority:
    - Critical: Immediate action required
    - Warning: Monitor closely, may need action
    - Healthy: No issues
    """
)

print("✅ System Specialist Agent created!")
print(f"   Model: {OPENAI_MODEL}")
print(f"   Tools: Connected to System Monitoring MCP Server")
print(f"   Capabilities: Health checks, metrics, logs")

---

## 🔌 Part 4: Connecting ADK Agents to MCP Servers

### The McpToolset Class

**Important**: You **do not** need to manually start MCP server processes!

When you create a `McpToolset`, ADK automatically:
1. Launches the MCP server process when needed
2. Establishes stdio communication
3. Discovers available tools
4. Manages the connection lifecycle
5. Cleans up when done

This is why we only need to specify the command and script path - ADK handles the rest!


**Key Components:**
- `McpToolset`: Main integration class (note the capitalization: Mcp not MCP)
- `StdioConnectionParams`: Connection configuration for stdio transport
- `StdioServerParameters`: Server launch parameters (command + args)
- `command`: Usually "python" for Python MCP servers, "npx" for Node.js servers
- `args`: List of arguments including the server script path




In [ ]:
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

# Create toolset for an MCP server
ticket_tools = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="python",
            args=["ticket_mcp_server.py"]
        )
    )
)

# Add to agent's tools
agent = LlmAgent(
    model=LiteLlm(model="openai/gpt-5-nano"),
    tools=[ticket_tools],  # Agent can now use all ticket DB tools!
    ...
)

---

## 🎭 Part 5: Building the Complete Multi-Agent System

Now let's create a coordinator agent that routes requests to our specialists!

In [ ]:
# Create IT Support Coordinator with all specialists
support_coordinator = LlmAgent(
    model=LiteLlm(model=f"openai/{OPENAI_MODEL}"),
    name="it_support_coordinator",
    sub_agents=[
        ticket_specialist,
        knowledge_specialist,
        system_specialist
    ],
    instruction="""
    You are the IT Support Coordinator.

    YOUR TEAM:
    You manage a team of specialists (these are your ONLY sub-agents):

    1. **ticket_specialist**: Manages support tickets
       - Use for: viewing, creating, updating tickets
       - Connected to ticket database via MCP
       - This specialist handles ALL ticket operations

    2. **knowledge_specialist**: Searches documentation
       - Use for: finding guides, troubleshooting steps
       - Connected to knowledge base via MCP

    3. **system_specialist**: Monitors system health
       - Use for: checking servers, diagnosing issues
       - Connected to monitoring system via MCP

    YOUR ROLE:
    1. Understand the user's request
    2. Route to the appropriate specialist(s)
    3. Coordinate multi-step workflows when needed
    4. Provide clear, helpful responses

    CRITICAL ROUTING RULES:
    - ONLY route to these three specialists: ticket_specialist, knowledge_specialist, system_specialist
    - NEVER try to route to team names (hardware_team, software_team, network_team, etc.)
    - When creating tickets, use ticket_specialist and pass team assignment as a parameter
    - Team names are DATA, not agents!

    ROUTING GUIDELINES:
    - Ticket questions → ticket_specialist
    - How-to questions → knowledge_specialist
    - System issues → system_specialist
    - Complex issues → multiple specialists in sequence

    WORKFLOW EXAMPLES:

    User: "What's the status of ticket T-1001?"
    → Route to ticket_specialist

    User: "How do I reset my password?"
    → Route to knowledge_specialist

    User: "Is the email server working?"
    → Route to system_specialist

    User: "Create a ticket for mouse issue, assign to hardware team"
    → Route to ticket_specialist (it will handle the team assignment)

    User: "Create a ticket for the email server issue"
    → 1. system_specialist (check email server)
    → 2. ticket_specialist (create ticket with details)

    Always be helpful, professional, and efficient.
    Remember: You can ONLY transfer to ticket_specialist, knowledge_specialist, or system_specialist!
    """
)

print("✅ IT Support Coordinator Agent created!")
print(f"   Model: {OPENAI_MODEL}")
print(f"   Sub-agents: 3 specialists")
print(f"   Capabilities: Complete IT support system")
print(f"\n🏗️  Architecture:")
print(f"   Coordinator (ADK) → Specialists (ADK) → MCP Servers → Data")

### 5.1: Setup Runner and Session

In [ ]:
# Create session service and runner
session_service = InMemorySessionService()
APP_NAME = "it_support_mcp_system"

runner = Runner(
    app_name=APP_NAME,
    agent=support_coordinator,
    session_service=session_service
)

print("✅ Runner initialized!")
print(f"   App: {APP_NAME}")
print(f"   Ready to handle IT support requests")

### 5.2: Helper Function for Testing

In [ ]:
# Track active sessions
_sessions = set()

async def ask_support_async(question: str, session_id: str = "support_session"):
    """
    Ask the IT support system a question.
    """
    user_id = "user_001"

    # Create session if needed
    session_key = (session_id, user_id)
    if session_key not in _sessions:
        await session_service.create_session(
            app_name=APP_NAME,
            user_id=user_id,
            session_id=session_id,
            state={}
        )
        _sessions.add(session_key)

    # Create message
    content = types.Content(role='user', parts=[types.Part(text=question)])

    print(f"\n{'='*80}")
    print(f"❓ USER QUESTION")
    print(f"{'='*80}")
    print(f"{question}")
    print(f"\n⏳ Processing (Coordinator → Specialists → MCP Servers)...\n")

    # Run agent
    events = runner.run_async(user_id=user_id, session_id=session_id, new_message=content)

    # Process events
    final_response = None
    async for event in events:
        if event.is_final_response():
            final_response = event.content.parts[0].text

    if final_response:
        print(f"\n{'='*80}")
        print(f"💬 SUPPORT RESPONSE")
        print(f"{'='*80}")
        print(final_response)
        print(f"{'='*80}\n")

    return final_response

def ask_support(question: str, session_id: str = "support_session"):
    """Synchronous wrapper."""
    try:
        loop = asyncio.get_running_loop()
        return asyncio.run(ask_support_async(question, session_id))
    except RuntimeError:
        return asyncio.run(ask_support_async(question, session_id))

print("✅ Helper function ready!")
print("   Use: ask_support('your question here')")

---

## 🎮 Part 6: Testing the Complete System

Let's test our complete ADK + MCP system with real scenarios!

### Test 1: Ticket Management

Testing the ticket specialist with MCP ticket database.

In [ ]:
# Ask about a specific ticket
ask_support("What's the status of ticket T-1001?")

In [ ]:
# List high priority tickets
ask_support("Show me all high priority tickets")

### Test 2: Knowledge Base Search

Testing the knowledge specialist with MCP knowledge base.

In [ ]:
# Search for password reset guide
ask_support("How do I reset my password?")

In [ ]:
# Search for WiFi troubleshooting
ask_support("My WiFi keeps disconnecting, what should I do?")

### Test 3: System Monitoring

Testing the system specialist with MCP monitoring server.

In [ ]:
# Check email server health
ask_support("Is the email server working? Check email-server-01")

In [ ]:
# Get all critical alerts
ask_support("Show me all critical system alerts")

### Test 4: Multi-Agent Workflow

Testing complex workflow requiring multiple specialists.

In [ ]:
# Complex request: check system + find docs + create ticket
ask_support(
    "The file server seems slow. Can you check its status, find troubleshooting docs, "
    "and create a ticket if there's an issue?"
)

### Test 5: Create New Ticket

Testing ticket creation via MCP.

In [ ]:
# Create a new ticket
ask_support(
    "Create a ticket: My mouse is not working properly. It keeps freezing. "
    "Priority: medium, assign to hardware team."
)

---

## 📊 Part 7: Architecture Overview

### What Just Happened?

Let's trace a request through the complete stack:

**Example: "What's the status of ticket T-1001?"**

```
1. USER
   ↓
   "What's the status of ticket T-1001?"
   ↓

2. COORDINATOR AGENT (ADK)
   ↓
   Uses OpenAI (gpt-5-nano) to understand request
   Decides: This is a ticket question
   Routes to: ticket_specialist
   ↓

3. TICKET SPECIALIST AGENT (ADK)
   ↓
   Has access to ticket_toolset (McpToolset)
   Uses OpenAI to decide: Need to call get_ticket tool
   ↓

4. McpToolset
   ↓
   Sends tool call to MCP server via stdio
   Tool: get_ticket(ticket_id="T-1001")
   ↓

5. TICKET MCP SERVER
   ↓
   Receives call_tool request
   Looks up ticket in database
   Returns ticket data as JSON
   ↓

6. McpToolset ← MCP SERVER
   ↓
   Receives JSON response
   Converts to ADK tool result
   ↓

7. TICKET SPECIALIST ← McpToolset
   ↓
   Receives tool result
   Uses OpenAI to format response
   ↓

8. COORDINATOR ← TICKET SPECIALIST
   ↓
   Receives specialist response
   Returns to user
   ↓

9. USER ← COORDINATOR
   ✅
   "Ticket T-1001: Laptop won't boot (Status: Open, Priority: High)"
```

### Layer Responsibilities

| Layer | Purpose | Technology | In Our System |
|-------|---------|------------|---------------|
| **Reasoning** | Understand intent, make decisions | OpenAI API | gpt-5-nano |
| **Orchestration** | Coordinate agents, manage workflows | Google ADK | Coordinator + Specialists |
| **Connectivity** | Standardized data access | MCP | 3 MCP servers |
| **Data** | Actual data storage | Various | In-memory dicts (demo) |

### Why This Architecture Wins

✅ **Modularity**
- MCP servers can be reused by any MCP client
- ADK agents can use any MCP server
- Can swap OpenAI for other providers

✅ **Scalability**
- Add new MCP servers without changing agents
- Add new specialists without changing coordinator
- Scale each layer independently

✅ **Maintainability**
- Clear separation of concerns
- Update data layer without touching AI logic
- Update AI logic without touching data layer

✅ **Composability**
- Mix and match components
- Build complex systems from simple pieces
- Reuse across projects

---

## 🎓 Part 8: Exercises

### Exercise 1: Create a New MCP Server (Intermediate)

**Task:** Create a simple User Directory MCP server.

**Requirements:**
1. Create `user_directory_mcp_server.py`
2. Store employee data (name, email, department, phone)
3. Implement tools:
   - `search_employee`: Find by name
   - `get_employee`: Get by email
   - `list_by_department`: Filter by department
4. Follow MCP server pattern from existing servers

**Bonus:** Connect it to the coordinator agent!

In [ ]:
# Exercise 1: Your code here

# TODO: Create user_directory_mcp_server.py
# TODO: Start the server
# TODO: Create McpToolset for user directory
# TODO: Create a user_directory_specialist agent
# TODO: Add to coordinator's sub_agents

### Exercise 2: Build a Report Generator (Advanced)

**Task:** Create an agent that generates IT support reports.

**Requirements:**
1. Create a `report_generator` agent
2. Access multiple MCP servers (tickets + systems)
3. Generate reports:
   - Open tickets summary
   - Critical system alerts
   - Team workload distribution
4. Format output nicely

**Hint:** Use multiple McpToolsets in one agent!

In [ ]:
# Exercise 2: Your code here

# TODO: Create report generator agent with multiple toolsets
# report_agent = LlmAgent(
#     model=LiteLlm(model=f"openai/{OPENAI_MODEL}"),
#     name="report_generator",
#     tools=[ticket_toolset, monitoring_toolset],  # Multiple MCP servers!
#     instruction="..."
# )

### Exercise 3: Implement Auto-Escalation (Advanced)

**Task:** Create workflow that auto-escalates critical issues.

**Requirements:**
1. Monitor for critical system alerts
2. Automatically create high-priority tickets
3. Search knowledge base for relevant docs
4. Include all info in ticket description

**Hint:** Use SequentialAgent or create custom workflow logic!

In [ ]:
# Exercise 3: Your code here

# TODO: Create auto-escalation workflow
# Steps:
# 1. Check for critical alerts (system_specialist)
# 2. For each alert, search KB (knowledge_specialist)
# 3. Create ticket with alert + KB info (ticket_specialist)
# 4. Consider using SequentialAgent or LoopAgent

---

## 🎯 Part 9: Summary

### What You've Learned

**Lesson 1: Foundations**
- Basic LLM agents with personality
- OpenAI integration via LiteLLM
- Session management

**Lesson 2: Tools**
- Function calling
- Custom tool development
- Tool best practices

**Lesson 3: Multi-Agent Coordination**
- Hierarchical routing
- Coordinator + specialists pattern
- sub_agents parameter

**Lesson 4: Workflow Patterns**
- SequentialAgent (pipelines)
- ParallelAgent (concurrent execution)
- Performance optimization

**Lesson 5: Advanced Patterns**
- Rule-based agents (BaseAgent)
- Hybrid architectures
- LoopAgent
- Cost optimization (70% savings!)

**Lesson 6: Complete Stack**
- MCP server development
- ADK + MCP integration
- Production architecture
- Composable systems



### Production Best Practices

✅ **Architecture**
- Separate concerns (reasoning, orchestration, data)
- Use MCP for data connectivity
- Design for composability

✅ **Cost Optimization**
- Rules first, LLM second (70% savings)
- Right-size models (gpt-5-nano vs gpt-4o)
- Cache when possible

✅ **Performance**
- Parallel execution when possible
- Async/await patterns
- Background processes for MCP servers

✅ **Maintainability**
- Clear agent responsibilities
- Reusable MCP servers
- Comprehensive instructions


### Resources

**ADK Documentation:**
- [ADK Official Docs](https://google.github.io/adk-docs/)
- [ADK MCP Integration](https://google.github.io/adk-docs/tools/mcp-tools/)
- [ADK Examples](https://github.com/google/adk-examples)

**MCP Documentation:**
- [MCP Specification](https://modelcontextprotocol.io/)
- [MCP Python SDK](https://github.com/modelcontextprotocol/python-sdk)
- [MCP Servers List](https://github.com/modelcontextprotocol/servers)

**Community:**
- [ADK Discord](https://discord.gg/google-adk)
- [MCP Community](https://github.com/modelcontextprotocol/community)



---

## 🧹 Part 10: Cleanup

Don't forget to stop MCP servers when done!

In [ ]:
# No manual cleanup needed!
# McpToolset automatically manages MCP server lifecycle

print("✅ Session completed!")
print("   MCP servers will be automatically cleaned up by ADK")
print("   Session data cleared from memory")